In [77]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split

In [78]:
import tensorflow.keras
import tensorflow as tf

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Reshape, Flatten, Conv1D, MaxPooling1D, Conv2D, MaxPooling2D, BatchNormalization, Dropout, Activation

# Read data From File

In [79]:
covid_df = pd.read_excel("corona value - 08-04-99.xlsx")
covid_df.columns = covid_df.iloc[1]
covid_df = covid_df.drop(covid_df.index[[0,1,2,3,4]]).reset_index().drop('index', axis=1)

In [80]:
covid_df = covid_df.dropna()
display(covid_df)
display(covid_df.describe())
display(covid_df.info())

1,Age (yr),Gender,pulse rate,respiratory rate,Tempreture,Height,Weight,BMI,Systolic blood presure,Diastolic blood pressure,...,non-corona chest CT signs,CT is requesyed,no need to CT study,HCQ,CAL,symptomatic therapy,AZT,OST,admission,No previous treatment
0,73,male,64,22,36.6,155,60,24.97,136,41,...,0,0,0,0,0,0,0,0,0,1
1,29,female,80,22,37.1,165,67,24.61,94,44,...,0,0,0,0,0,0,0,0,0,1
4,34,female,102,22,37.2,170,53,18.34,114,47,...,0,0,0,0,0,0,0,0,0,1
5,19,male,124,20,36.9,170,65,22.49,123,50,...,0,0,1,0,0,0,0,0,0,1
6,13,female,89,18,36.3,155,54,22.48,115,51,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3474,52,female,120,19,36,173,85,28.4,156,110,...,0,1,0,0,0,0,0,0,0,1
3475,75,female,92,18,37,160,50,19.53,183,117,...,0,0,0,0,0,0,0,0,0,1
3477,39,female,125,23,37,162,106,40.39,185,125,...,0,0,0,0,0,0,0,0,0,0
3478,56,male,150,24,35,168,65,23.03,231,148,...,0,0,1,0,0,1,0,0,0,0


1,Age (yr),Gender,pulse rate,respiratory rate,Tempreture,Height,Weight,BMI,Systolic blood presure,Diastolic blood pressure,...,non-corona chest CT signs,CT is requesyed,no need to CT study,HCQ,CAL,symptomatic therapy,AZT,OST,admission,No previous treatment
count,2577,2577,2577,2577,2577.0,2577,2577,2577.00,2577,2577,...,2577,2577,2577,2577,2577,2577,2577,2577,2577,2577
unique,83,2,99,22,48.0,55,91,825.00,117,90,...,1,2,2,2,2,2,2,2,2,2
top,34,male,100,20,36.5,170,80,27.68,120,80,...,0,0,0,0,0,0,0,0,0,0
freq,98,1418,128,1079,279.0,318,209,54.00,96,114,...,2577,2198,2057,2216,2568,2221,2404,2545,2514,1573


<class 'pandas.core.frame.DataFrame'>
Int64Index: 2577 entries, 0 to 3479
Columns: 102 entries, Age (yr) to No previous treatment
dtypes: object(102)
memory usage: 2.0+ MB


None

In [81]:
cols=[i for i in covid_df.columns if i not in ["Gender"]]
covid_df['Triage result'] = covid_df['Triage result'].astype(str)
for col in cols:
    covid_df[col]=pd.to_numeric(covid_df[col])
    covid_df[col]=covid_df[col].astype(np.float64)
display(covid_df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2577 entries, 0 to 3479
Columns: 102 entries, Age (yr) to No previous treatment
dtypes: float64(101), object(1)
memory usage: 2.0+ MB


None

In [9]:
from sklearn import preprocessing   
label_encoder = preprocessing.LabelEncoder() 
covid_df['Gender']= label_encoder.fit_transform(covid_df['Gender']) 
covid_df['Gender'] = covid_df['Gender'].astype(np.float64)

In [10]:
covid_df = covid_df.drop(covid_df.loc[:,'Lymp<1100':].columns, axis=1)

In [11]:
target = covid_df['Triage result']
train = covid_df.drop('Triage result', axis=1)

x_train, x_test, y_train, y_test = train_test_split(train, target, test_size=0.2, stratify=target, random_state=14)

print("Test: Number of each Class: " , sorted(Counter(y_test).items()), " Number of Samples: " ,len(x_test))
print("Train: Number of each Class: " , sorted(Counter(y_train).items()), " Number of Samples: " ,len(x_train))

Test: Number of each Class:  [(1.0, 225), (2.0, 261), (3.0, 30)]  Number of Samples:  516
Train: Number of each Class:  [(1.0, 899), (2.0, 1041), (3.0, 121)]  Number of Samples:  2061


In [86]:
ros = RandomOverSampler(random_state=0)
x_train_ros, y_train_ros = ros.fit_resample(x_train, y_train)
print(sorted(Counter(y_train_ros).items()))

[(1.0, 1041), (2.0, 1041), (3.0, 1041)]


In [88]:
print(x_train_ros.shape)
# x_train = x_train.values.reshape(2061, 78, 1)
# x_train.shape

(3123, 78)


In [87]:
dataset = tf.data.Dataset.from_tensor_slices((x_train_ros.values, y_train_ros.values))
# for feat, targ in dataset.take(5):
#   print ('Features: {}, Target: {}'.format(feat, targ))

In [89]:
train_dataset = dataset.shuffle(len(x_train_ros)).batch(1)

In [90]:
def createCNN(width, features, classes):
    model = Sequential()
    inputShape = (102, classes)
    model.add(Reshape((1,x_train.shape[1],1)))
    model.add(Conv2D(filters=8, kernel_size=(5, 5), input_shape=(1,x_train.shape[1],1), activation="relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(filters=16, kernel_size=(3, 3), activation="relu"))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=16, kernel_size=(3, 3), activation="relu"))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(filters=32, kernel_size=(3, 3), padding="same", activation="relu"))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=32, kernel_size=(3, 3), padding="same", activation="relu"))
    model.add(BatchNormalization())

    model.add(Flatten())
    model.add(Dropout(0.5))
    model.add(Dense(512, activation="relu"))
    model.add(Dense(1, activation="softmax"))
    return model

In [91]:
learning_rate = 0.001
epochs = 15
batch_size = 64
optimizer = Adam(lr=learning_rate, decay=learning_rate / (epochs))

my_model = createCNN(2577, 78, 3)


In [94]:
my_model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])
my_model.build((1,x_train.shape[1],1))
my_model.summary()

ValueError: Negative dimension size caused by subtracting 5 from 1 for '{{node conv2d_10/Conv2D}} = Conv2D[T=DT_FLOAT, data_format="NHWC", dilations=[1, 1, 1, 1], explicit_paddings=[], padding="VALID", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true](reshape_8/Reshape, conv2d_10/Conv2D/ReadVariableOp)' with input shapes: [1,1,78,1], [5,5,1,8].

In [95]:
my_model_fit = my_model.fit(train_dataset, batch_size=batch_size, epochs=epochs, verbose=1)

Epoch 1/15

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



ValueError: in user code:

    /home/sspc/anaconda3/envs/DecisionTree_MashinLearning/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:806 train_function  *
        return step_function(self, iterator)
    /home/sspc/anaconda3/envs/DecisionTree_MashinLearning/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /home/sspc/anaconda3/envs/DecisionTree_MashinLearning/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/sspc/anaconda3/envs/DecisionTree_MashinLearning/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/sspc/anaconda3/envs/DecisionTree_MashinLearning/lib/python3.7/site-packages/tensorflow/python/distr